<a href="https://colab.research.google.com/github/HanifAmeen/SE-44-Dermorax/blob/ML-Model---InceptionResNet-V2%2BSoft-Attention/SDGP_CNN_Model_IRV2%2BSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Mounting the drive

In [1]:
#mounting the drive 
from google.colab import drive
drive.mount('/content/drive')
print("done")

Mounted at /content/drive
done


#Imports

In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#Initializing the dataset

In [4]:
data_pd = pd.read_csv('/content/drive/MyDrive/Dataset/HAM10000_metadata')
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [5]:
#counting the images per each class
def count_images_on_type(series):
    display(pd.DataFrame(series.value_counts()))

class_list = ['akiec','bcc','bkl','df','mel','nv','vasc']
count_images_on_type(data_pd['dx'])


,dx
nv,6705
mel,1113
bkl,1099
bcc,514
akiec,327
vasc,142
df,115


In [6]:
train_dir = os.path.join('HAM10000', 'train_dir')

test_dir = os.path.join('HAM10000', 'test_dir')


In [6]:
#making the directories to store test and train data
os.mkdir('/content/drive/MyDrive/Dataset/train_dir')
os.mkdir('/content/drive/MyDrive/Dataset/test_dir')

In [7]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()

,image_id,dx,dx_type,age,sex,localization,dataset
lesion_id,,,,,,,
HAM_0000000,2,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1,1


In [8]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)
df_count.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000001,1,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1,1


In [9]:
#finding the duplicates to make sure none of these images have augmented duplicates

def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicate'

data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicate


In [10]:
#framing all the meta data
df_count = data_pd[data_pd['is_duplicate'] == 'no']
print(df_count)

         lesion_id      image_id     dx  ... localization       dataset is_duplicate
10     HAM_0001396  ISIC_0025276    bkl  ...        trunk  vidir_modern           no
15     HAM_0007207  ISIC_0031326    bkl  ...         back  vidir_modern           no
20     HAM_0006071  ISIC_0032343    bkl  ...         face  vidir_modern           no
33     HAM_0005612  ISIC_0024981    bkl  ...        scalp  vidir_modern           no
34     HAM_0005388  ISIC_0027815    bkl  ...        chest     rosendahl           no
...            ...           ...    ...  ...          ...           ...          ...
9988   HAM_0001036  ISIC_0027588  akiec  ...         face     rosendahl           no
9990   HAM_0004462  ISIC_0027334  akiec  ...        trunk  vidir_modern           no
9991   HAM_0001152  ISIC_0030133  akiec  ...         face     rosendahl           no
10001  HAM_0000020  ISIC_0031922  akiec  ...         face     rosendahl           no
10008  HAM_0001576  ISIC_0033705  akiec  ...         face  vidir_

#Test Train Split 15% to 85%

In [11]:
train, test_df = train_test_split(df_count, test_size=0.15, stratify=df_count['dx'])

In [12]:
#labels the data as test or train
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'


In [13]:
test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
823,HAM_0003514,ISIC_0030369,bkl,confocal,70.0,male,face,vidir_modern,no
2627,HAM_0002559,ISIC_0034047,bcc,histo,70.0,male,face,vidir_modern,no
6219,HAM_0002292,ISIC_0031183,nv,follow_up,30.0,female,abdomen,vidir_molemax,no
5939,HAM_0004041,ISIC_0026375,nv,follow_up,50.0,female,foot,vidir_molemax,no
9961,HAM_0001023,ISIC_0026468,akiec,histo,50.0,male,hand,rosendahl,no


In [14]:
#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicate,train


In [15]:
# adding to lists by image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [16]:
len(train_list)

9187

In [17]:
len(test_list)

828

In [18]:
# Set the image_id as the index in data_pd
data_pd.set_index('image_id', inplace=True)

In [19]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [20]:
#making the directories per each cancer type in test and train dir 
for i in targetnames:
  os.mkdir("/content/drive/MyDrive/Dataset/test_dir/"+i)
  os.mkdir("/content/drive/MyDrive/Dataset/train_dir/"+i)

##Copying the images to the test and the train folders

In [21]:
#compying images to train folders
for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('/content/drive/MyDrive/Dataset/HAM10000/',file_name)

    # copying the image from the source to target file
    target = os.path.join('/content/drive/MyDrive/Dataset/train_dir/',label,file_name) 

    shutil.copyfile(source, target)

In [22]:
#copying images to the test folders
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('/content/drive/MyDrive/Dataset/HAM10000/',file_name)

    # copying the image from the source to target file
    target = os.path.join('/content/drive/MyDrive/Dataset/test_dir/',label,file_name)

    shutil.copyfile(source, target)

##Image Argumentation

In [23]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

# Augmenting images and storing them in temporary directories 
for img_class in targetnames:

    #creating temporary directories
    # creating a base directory
    
    os.mkdir('/content/drive/MyDrive/Dataset/aug_dir')
    # creating a subdirectory inside the base directory for images of the same class
    img_dir = os.path.join('aug_dir', 'img_dir')
    os.mkdir('/content/drive/MyDrive/Dataset/aug_dir/img_dir')

    img_list = os.listdir('/content/drive/MyDrive/Dataset/train_dir/' + img_class)

    # Copy images from the class train_dir to the img_dir 
    for file_name in img_list:

        # path of source image in training directory
        source = os.path.join('/content/drive/MyDrive/Dataset/train_dir/',img_class, file_name)

        # creating a target directory to send images 
        target = os.path.join('/content/drive/MyDrive/Dataset/aug_dir/img_dir/',file_name)

        # copying the image from the source to target file
        shutil.copyfile(source, target)

    # Temporary augumented dataset directory.
    source_path = '/content/drive/MyDrive/Dataset/aug_dir/'

    # Augmented images will be saved to training directory
    save_path = '/content/drive/MyDrive/Dataset/train_dir/'+img_class

    # Creating Image Data Generator to augment images
    # Rotating,flipping and shifting the images to create augmented images.
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(

        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'

    )

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(source_path,
                                              save_to_dir=save_path,
                                              save_format='jpg',
                                              target_size=(224, 224),
                                              batch_size=batch_size)

    # Generate the augmented images
    aug_images = 8000 

    num_files = len(os.listdir('/content/drive/MyDrive/Dataset/aug_dir/img_dir/'))
    num_batches = int(np.ceil((aug_images - num_files) / batch_size))

    # creating 8000 augmented images per class
    for i in range(0, num_batches):
        images, labels = next(aug_datagen)

    # delete temporary directory 
    shutil.rmtree('/content/drive/MyDrive/Dataset/aug_dir')


Found 304 images belonging to 1 classes.
Found 488 images belonging to 1 classes.
Found 1033 images belonging to 1 classes.
Found 109 images belonging to 1 classes.
Found 1079 images belonging to 1 classes.
Found 6042 images belonging to 1 classes.
Found 132 images belonging to 1 classes.


In [20]:
train_path = '/content/drive/MyDrive/Dataset/train_dir'
test_path = '/content/drive/MyDrive/Dataset/test_dir'
batch_size = 16

In [21]:
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

In [22]:
#To find the total number of images in the training batch and test batch.
image_size = 299
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 51699 images belonging to 7 classes.

Test Batches: 
Found 828 images belonging to 7 classes.


#Model



##Soft Attention Layer

In [23]:
#initializing the Soft-Attention layer

class SoftAttention(Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x

        
        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC
    
        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]
        
        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape
        

        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = K.expand_dims(x,axis=-1)

        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)

        conv3d = K.permute_dimensions(conv3d,pattern=(0,4,1,2,3))

        
        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = K.softmax(conv3d, axis=-1) 
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)

        
        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)       
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))
   
            x_exp = K.expand_dims(x,axis=-2)
   
            u = kl.Multiply()([exp_softmax_alpha, x_exp])   
  
            u = kl.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = K.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)

            u = kl.Multiply()([exp_softmax_alpha, x])   

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u,x])
        else:
            o = u
        
        return [o, softmax_alpha]

 #Finalizing the Soft-Attention layer       

    def compute_output_shape(self, input_shape): 
        return [self.out_features_shape, self.out_attention_maps_shape]

    
    def get_config(self):
        return super(SoftAttention,self).get_config()

##Downloading the InceptionResNetV2 Model

In [24]:
irv2 = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classifier_activation="softmax",

)

# Excluding the last 28 layers of the model.
#To insert the Soft-Attention layer.

conv = irv2.layers[-28].output

225222656/225209952 [==============================] - 1s 0us/step


##Combining the Soft-Attention layer to the IRV2

In [25]:
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(conv.shape[-1]),name='soft_attention')(conv)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
conv=(MaxPooling2D(pool_size=(2, 2),padding="same")(conv))

conv = concatenate([conv,attention_layer])
conv  = Activation('relu')(conv)
conv = Dropout(0.5)(conv)

In [26]:
output = Flatten()(conv)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=irv2.input, outputs=output)

#Compiling the model

In [27]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.01,epsilon=0.1)
model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Model Summary

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

#Model Evaluation Before Training

In [31]:
batch_size=16
#taking predictions from the trained model against the test images
predictions = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

In [32]:
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
#getting the true labels per image 
y_true = test_batches.classes



# Creating classification report 
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report BEFORE TRAINING for all 7 calsses :")
print("\n")
print(report)


Classification Report BEFORE TRAINING for all 7 calsses :


              precision    recall  f1-score   support

       akiec       0.00      0.00      0.00        23
         bcc       0.02      0.04      0.02        26
         bkl       0.15      0.15      0.15        66
          df       0.01      0.17      0.01         6
         mel       0.08      0.18      0.11        34
          nv       0.81      0.54      0.65       663
        vasc       0.00      0.00      0.00        10

    accuracy                           0.46       828
   macro avg       0.15      0.15      0.14       828
weighted avg       0.67      0.46      0.54       828



#Training the model

In [33]:
class_weights = {   
                    0 : 1.0,  ## akiec
                    1 : 1.0,  ## bcc
                    2 : 1.0,  ## bkl
                    3 : 1.0,  ## df
                    4 : 5.0,  ## mel
                    5 : 1.0,  ## nv
                    6 : 1.0,  ## vasc
                }

In [34]:
#checkpoints to save the weights of the model after every epoch
checkpoint_path ='/content/drive/MyDrive/Dataset/IRV2_SA_CP.cpkt'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [35]:
#Reloading the model weights from the IRV2_SA_CP.spkt file
model.load_weights(checkpoint_path)

In [51]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=2, min_delta=0.001)

#training the model
history = model.fit(train_batches,
                    steps_per_epoch=(len(train_df)/10),
                    epochs=10,
                    batch_size=32,
                    validation_data=test_batches,
                    validation_steps=len(test_df)/batch_size,
                    callbacks=[cp_callback,Earlystop],
                    class_weight=class_weights)

Epoch 1/10
919/918 [==============================] - ETA: 0s - loss: 0.7515 - accuracy: 0.7873
Epoch 1: saving model to /content/drive/MyDrive/Dataset/IRV2_SA_CP.cpkt
918/918 [==============================] - 1191s 1s/step - loss: 0.7515 - accuracy: 0.7873 - val_loss: 0.3786 - val_accuracy: 0.8478
Epoch 2/10
919/918 [==============================] - ETA: 0s - loss: 0.6899 - accuracy: 0.8017
Epoch 2: saving model to /content/drive/MyDrive/Dataset/IRV2_SA_CP.cpkt
918/918 [==============================] - 1191s 1s/step - loss: 0.6899 - accuracy: 0.8017 - val_loss: 0.4445 - val_accuracy: 0.8430
Epoch 3/10
919/918 [==============================] - ETA: 0s - loss: 0.6294 - accuracy: 0.8251
Epoch 3: saving model to /content/drive/MyDrive/Dataset/IRV2_SA_CP.cpkt
918/918 [==============================] - 1185s 1s/step - loss: 0.6294 - accuracy: 0.8251 - val_loss: 0.3708 - val_accuracy: 0.8659
Epoch 4/10
919/918 [==============================] - ETA: 0s - loss: 0.5584 - accuracy: 0.8424
E

In [39]:
#code to save the model
model.save('/content/drive/MyDrive/Dataset/IRV2_SA_MODEL.hdf5')

##Model Evaluation after Training

In [36]:
batch_size=16
#taking predictions from the trained model against the test images
predictions = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)


In [38]:
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
#getting the true labels per image 
y_true = test_batches.classes


# Creating classification report 
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report AFTER TRAINING for all 7 calsses :")
print(report)


Classification Report AFTER TRAINING for all 7 calsses :
              precision    recall  f1-score   support

       akiec       0.55      0.52      0.53        23
         bcc       0.69      0.77      0.73        26
         bkl       0.77      0.45      0.57        66
          df       1.00      0.33      0.50         6
         mel       0.23      0.85      0.36        34
          nv       0.97      0.88      0.92       663
        vasc       1.00      0.90      0.95        10

    accuracy                           0.83       828
   macro avg       0.74      0.67      0.65       828
weighted avg       0.90      0.83      0.85       828

